# Match-TTSG: UNIFIED SPEECH AND GESTURE SYNTHESIS USING FLOW MATCHING
---
[Shivam Mehta](https://www.kth.se/profile/smehta), [Ruibo Tu](https://www.kth.se/profile/ruibo), [Simon Alexanderson
](https://www.kth.se/profile/simonal?l=en) [Jonas Beskow](https://www.kth.se/profile/beskow), [Éva Székely](https://www.kth.se/profile/szekely), and [Gustav Eje Henter](https://people.kth.se/~ghe/)

As text-to-speech technologies achieve remarkable naturalness in read-aloud tasks, there is growing interest in multimodal synthesis of verbal and non-verbal communicative behaviour, such as spontaneous speech and associated body gestures. This paper presents a novel, unified architecture for jointly synthesising speech acoustics and skeleton-based 3D gesture motion from text, trained using optimal-transport conditional flow matching (OT-CFM). The proposed architecture is simpler than the previous state of the art, has a smaller memory footprint, and can capture the joint distribution of speech and gestures, generating both modalities together in one single process. The new training regime, meanwhile, enables better synthesis quality in much fewer steps (network evaluations) than before. Uni- and multimodal subjective tests demonstrate improved speech naturalness, gesture human-likeness, and cross-modal appropriateness compared to existing benchmarks.

Demo Page: https://shivammehta25.github.io/Match-TTSG \
Code: https://github.com/shivammehta25/Match-TTSG




In [1]:
%env CUDA_VISIBLE_DEVICES=0

env: CUDA_VISIBLE_DEVICES=0


In [9]:
import datetime as dt
import warnings
from pathlib import Path

import ffmpeg
import IPython.display as ipd
import joblib as jl
import numpy as np
import soundfile as sf
import torch
from tqdm.auto import tqdm

# Hifigan imports
from match_ttsg.hifigan.config import v1
from match_ttsg.hifigan.denoiser import Denoiser
from match_ttsg.hifigan.env import AttrDict
from match_ttsg.hifigan.models import Generator as HiFiGAN
# MatchTTSG imports
from match_ttsg.models.match_ttsg_custom import MatchTTSGCustom
from match_ttsg.text import sequence_to_text, text_to_sequence
from match_ttsg.utils.model import denormalize
from match_ttsg.utils.utils import get_user_data_dir, intersperse

# Motion visualisation imports
from pymo.preprocessing import MocapParameterizer
from pymo.viz_tools import render_mp4
from pymo.writers import BVHWriter

In [10]:
%load_ext autoreload
%autoreload 2
%matplotlib inline
# This allows for real time code changes being reflected in the notebook, no need to restart the kernel

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [11]:
print(torch.cuda.is_available())
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

True


## Filepaths

In [19]:
MATCHTTSG_CHECKPOINT = "logs/train/multimodal_single_abi_baseline/runs/2024-12-12_15-40-39/checkpoints/last.ckpt"
#MOTION_PIPELINE = "match_ttsg/resources/data_pipe.expmap_86.1328125fps.sav"
HIFIGAN_CHECKPOINT = get_user_data_dir() / "generator_v1"#"hifigan_T2_v1"
OUTPUT_FOLDER = "synth_output"

print(HIFIGAN_CHECKPOINT)

/home/ayrton/.local/share/matcha_tts/generator_v1


## Load Match-TTSG

In [20]:
def load_model(checkpoint_path):
    model = MatchTTSGCustom.load_from_checkpoint(checkpoint_path, map_location=device)
    model.eval()
    return model
count_params = lambda x: f"{sum(p.numel() for p in x.parameters()):,}"


model = load_model(MATCHTTSG_CHECKPOINT)
print(f"Model loaded! Parameter count: {count_params(model)}")

Model loaded! Parameter count: 18,448,463


## Load HiFi-GAN (Vocoder)

In [21]:
def load_vocoder(checkpoint_path):
    h = AttrDict(v1)
    hifigan = HiFiGAN(h).to(device)
    hifigan.load_state_dict(torch.load(checkpoint_path, map_location=device)['generator'])
    _ = hifigan.eval()
    hifigan.remove_weight_norm()
    return hifigan

vocoder = load_vocoder(HIFIGAN_CHECKPOINT)
denoiser = Denoiser(vocoder, mode='zeros')

/home/ayrton/anaconda3/envs/match-ttsg-wsl/lib/python3.10/site-packages/torch/nn/utils/weight_norm.py:143: FutureWarning: `torch.nn.utils.weight_norm` is deprecated in favor of `torch.nn.utils.parametrizations.weight_norm`.
  WeightNorm.apply(module, name, dim)
/tmp/ipykernel_3031247/977743176.py:4: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `we

Removing weight norm...


# Load motion visualiser

In [33]:
motion_pipeline = jl.load(MOTION_PIPELINE)
bvh_writer = BVHWriter()
mocap_params = MocapParameterizer("position")

/home/smehta/anaconda3/envs/match_ttsg_repo/lib/python3.10/site-packages/sklearn/base.py:376: InconsistentVersionWarning: Trying to unpickle estimator Pipeline from version 0.24.2 when using version 1.4.1.post1. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(


### Helper functions to synthesise

In [43]:
@torch.inference_mode()
def process_text(text: str):
    #x = torch.tensor(intersperse(text_to_sequence(text, ['japanese_accent_cleaners']), 0),dtype=torch.long, device=device)[None]
    x = torch.tensor(text_to_sequence(text, ['japanese_accent_cleaners']),dtype=torch.long, device=device)[None]
    x_lengths = torch.tensor([x.shape[-1]],dtype=torch.long, device=device)
    x_phones = sequence_to_text(x.squeeze(0).tolist())
    return {
        'x_orig': text,
        'x': x,
        'x_lengths': x_lengths,
        'x_phones': x_phones
    }


@torch.inference_mode()
def synthesise(text, spks=None):
    text_processed = process_text(text)
    start_t = dt.datetime.now()
    output = model.synthesise(
        text_processed['x'], 
        text_processed['x_lengths'],
        n_timesteps=n_timesteps,
        temperature=temperature,
        spks=spks,
        length_scale=length_scale
    )
    # merge everything to one dict    
    output.update({'start_t': start_t, **text_processed})
    return output

@torch.inference_mode()
def to_waveform(mel, vocoder):
    audio = vocoder(mel).clamp(-1, 1)
    audio = denoiser(audio.squeeze(0), strength=0.00025).cpu().squeeze()
    return audio.cpu().squeeze()

"""
def to_bvh(motion):
    with warnings.catch_warnings():
        warnings.simplefilter("ignore")
        return motion_pipeline.inverse_transform([motion.cpu().squeeze(0).T])
"""  
def save_to_folder(filename: str, output: dict, folder: str):
    folder = Path(folder)
    folder.mkdir(exist_ok=True, parents=True)
    np.save(folder / f'{filename}', output['mel'].cpu().numpy())
    sf.write(folder / f'{filename}.wav', output['waveform'], 22050, 'PCM_24')
    #with open(folder / f'{filename}.bvh', 'w') as f:
        #bvh_writer.write(output['bvh'], f)
"""
def to_stick_video(filename, bvh, folder):
    folder = Path(folder)
    folder.mkdir(exist_ok=True, parents=True)

    with warnings.catch_warnings():
        warnings.simplefilter("ignore")
        X_pos = mocap_params.fit_transform([bvh])
    print(f"rendering {filename} ...")
    render_mp4(X_pos[0], folder / f'{filename}.mp4', axis_scale=200)


def combine_audio_video(filename: str, folder: str):
    print("Combining audio and video")
    folder = Path(folder)
    folder.mkdir(exist_ok=True, parents=True)

    input_video = ffmpeg.input(str(folder / f'{filename}.mp4'))
    input_audio = ffmpeg.input(str(folder / f'{filename}.wav'))
    output_filename = folder / f'{filename}_audio.mp4'
    ffmpeg.concat(input_video, input_audio, v=1, a=1).output(str(output_filename)).run(overwrite_output=True)
    print(f"Final output with audio: {output_filename}")
"""

'\ndef to_stick_video(filename, bvh, folder):\n    folder = Path(folder)\n    folder.mkdir(exist_ok=True, parents=True)\n\n    with warnings.catch_warnings():\n        warnings.simplefilter("ignore")\n        X_pos = mocap_params.fit_transform([bvh])\n    print(f"rendering {filename} ...")\n    render_mp4(X_pos[0], folder / f\'{filename}.mp4\', axis_scale=200)\n\n\ndef combine_audio_video(filename: str, folder: str):\n    print("Combining audio and video")\n    folder = Path(folder)\n    folder.mkdir(exist_ok=True, parents=True)\n\n    input_video = ffmpeg.input(str(folder / f\'{filename}.mp4\'))\n    input_audio = ffmpeg.input(str(folder / f\'{filename}.wav\'))\n    output_filename = folder / f\'{filename}_audio.mp4\'\n    ffmpeg.concat(input_video, input_audio, v=1, a=1).output(str(output_filename)).run(overwrite_output=True)\n    print(f"Final output with audio: {output_filename}")\n'

In [44]:
print(process_text("よろしくお願いします"))

{'x_orig': 'よろしくお願いします', 'x': tensor([[ 8, 62, 52,  4, 55, 52, 56, 45, 20, 48, 58, 52, 51, 42, 44, 38, 46, 56,
         45, 46, 50, 38,  5, 56, 32,  7]], device='cuda:0'), 'x_lengths': tensor([26], device='cuda:0'), 'x_phones': ':yo(roshIkuonegaishima)sU.'}


## Setup text to synthesise

In [49]:
texts = [
    "今日はいい天気ですね"
]

### Hyperparameters

In [46]:
## Number of ODE Solver steps
n_timesteps = 500

## Changes to the speaking rate
length_scale=1.0

## Sampling temperature
temperature = 0.667

## Synthesis

In [47]:
from match_ttsg.utils.model import denormalize
mel_mean = -5.264796733856201
mel_std = 1.5986379384994507

In [50]:
outputs, rtfs = [], []
rtfs_w = []
for i, text in enumerate(tqdm(texts)):
    output = synthesise(text) #, torch.tensor([15], device=device, dtype=torch.long).unsqueeze(0))
    #output['waveform'] = to_waveform(output['mel'], vocoder)
    output['waveform'] = to_waveform(denormalize(output['decoder_outputs_mel'], mel_mean, mel_std), vocoder)
    #output['bvh'] = to_bvh(output['motion'])[0]

    # Compute Real Time Factor (RTF) with HiFi-GAN
    t = (dt.datetime.now() - output['start_t']).total_seconds()
    rtf_w = t * 22050 / (output['waveform'].shape[-1])

    ## Pretty print
    print(f"{'*' * 53}")
    print(f"Input text - {i}")
    print(f"{'-' * 53}")
    print(output['x_orig'])
    print(f"{'*' * 53}")
    print(f"Phonetised text - {i}")
    print(f"{'-' * 53}")
    print(output['x_phones'])
    print(f"{'*' * 53}")
    print(f"RTF:\t\t{output['rtf']:.6f}")
    print(f"RTF Waveform:\t{rtf_w:.6f}")
    rtfs.append(output['rtf'])
    rtfs_w.append(rtf_w)
    outputs.append(output)

    ## Display the synthesised waveform
    ipd.display(ipd.Audio(output['waveform'], rate=22050))

    ## Save the generated waveform
    save_to_folder(i, output, OUTPUT_FOLDER)

print(f"Number of ODE steps: {n_timesteps}")
print(f"Mean RTF:\t\t\t\t{np.mean(rtfs):.6f} ± {np.std(rtfs):.6f}")
print(f"Mean RTF Waveform (incl. vocoder):\t{np.mean(rtfs_w):.6f} ± {np.std(rtfs_w):.6f}")

  0%|          | 0/1 [00:00<?, ?it/s]

*****************************************************
Input text - 0
-----------------------------------------------------
今日はいい天気ですね
*****************************************************
Phonetised text - 0
-----------------------------------------------------
:kyo)owa i)i te)NkidesUne.
*****************************************************
RTF:		3.304738
RTF Waveform:	3.346854


Number of ODE steps: 500
Mean RTF:				3.304738 ± 0.000000
Mean RTF Waveform (incl. vocoder):	3.346854 ± 0.000000


In [50]:
for i, output in enumerate(tqdm(outputs)):
    to_stick_video(i, output['bvh'], OUTPUT_FOLDER)
    combine_audio_video(i, OUTPUT_FOLDER)

  0%|          | 0/1 [00:00<?, ?it/s]

MocapParameterizer: position
rendering 0 ...


/home/smehta/Projects/Match-TTSG/pymo/viz_tools.py:168: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  parent_x = data["%s_Xposition" % joint][frame]
/home/smehta/Projects/Match-TTSG/pymo/viz_tools.py:169: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  parent_y = data["%s_Yposition" % joint][frame]
/home/smehta/Projects/Match-TTSG/pymo/viz_tools.py:170: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  parent_z = data["%s_Zposition" % joi

Combining audio and video


ffmpeg version 4.2.7-0ubuntu0.1 Copyright (c) 2000-2022 the FFmpeg developers
  built with gcc 9 (Ubuntu 9.4.0-1ubuntu1~20.04.1)
  configuration: --prefix=/usr --extra-version=0ubuntu0.1 --toolchain=hardened --libdir=/usr/lib/x86_64-linux-gnu --incdir=/usr/include/x86_64-linux-gnu --arch=amd64 --enable-gpl --disable-stripping --enable-avresample --disable-filter=resample --enable-avisynth --enable-gnutls --enable-ladspa --enable-libaom --enable-libass --enable-libbluray --enable-libbs2b --enable-libcaca --enable-libcdio --enable-libcodec2 --enable-libflite --enable-libfontconfig --enable-libfreetype --enable-libfribidi --enable-libgme --enable-libgsm --enable-libjack --enable-libmp3lame --enable-libmysofa --enable-libopenjpeg --enable-libopenmpt --enable-libopus --enable-libpulse --enable-librsvg --enable-librubberband --enable-libshine --enable-libsnappy --enable-libsoxr --enable-libspeex --enable-libssh --enable-libtheora --enable-libtwolame --enable-libvidstab --enable-libvorbis --e

Final output with audio: synth_output/0_audio.mp4


frame= 2268 fps=450 q=-1.0 Lsize=    1059kB time=00:00:18.90 bitrate= 459.2kbits/s speed=3.75x    
video:846kB audio:177kB subtitle:0kB other streams:0kB global headers:0kB muxing overhead: 3.593406%
[aac @ 0x563432dd2740] Qavg: 2339.779
[libx264 @ 0x563432dd3e80] frame I:10    Avg QP:15.14  size: 49158
[libx264 @ 0x563432dd3e80] frame P:572   Avg QP:21.38  size:   491
[libx264 @ 0x563432dd3e80] frame B:1686  Avg QP:19.99  size:    55
[libx264 @ 0x563432dd3e80] consecutive B-frames:  0.8%  0.1%  0.1% 98.9%
[libx264 @ 0x563432dd3e80] mb I  I16..4: 52.5% 21.3% 26.2%
[libx264 @ 0x563432dd3e80] mb P  I16..4:  0.0%  0.0%  0.0%  P16..4:  0.1%  0.2%  0.3%  0.0%  0.0%    skip:99.4%
[libx264 @ 0x563432dd3e80] mb B  I16..4:  0.0%  0.0%  0.0%  B16..8:  0.1%  0.0%  0.0%  direct: 0.0%  skip:99.9%  L0:27.2% L1:58.9% BI:13.9%
[libx264 @ 0x563432dd3e80] 8x8 transform intra:21.3% inter:17.5%
[libx264 @ 0x563432dd3e80] coded y,uvDC,uvAC intra: 21.3% 1.4% 1.2% inter: 0.1% 0.1% 0.1%
[libx264 @ 0x563432dd3